##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Migrar de Estimator a las API de Keras

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/migrating_estimator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/migrating_estimator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/migrating_estimator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/migrate/migrating_estimator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
</table>

Esta guía muestra cómo migrar de las API `tf.estimator.Estimator` de TensorFlow 1 a las API `tf.keras` de TensorFlow 2. En primer lugar, prepararás y ejecutarás un modelo básico de entrenamiento y evaluación con `tf.estimator.Estimator`. A continuación, realizarás los pasos equivalentes en TensorFlow 2 con las API `tf.keras`. También aprenderás a personalizar el paso de entrenamiento subclasificando `tf.keras.Model` y usando `tf.GradientTape`.

- En TensorFlow 1, las API de alto nivel `tf.estimator.Estimator` te permiten entrenar y evaluar un modelo, así como realizar inferencias y guardar tu modelo (para servirlo).
- En TensorFlow 2, usa las APIs Keras para realizar las tareas mencionadas, como [construcción de modelos](https://www.tensorflow.org/guide/keras/custom_layers_and_models), aplicación de gradientes, [entrenamiento](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit), evaluación y predicción.

(Para migrar flujos de trabajo de guardado de modelos/puntos de verificación a TensorFlow 2, consulta las guías de migración [SavedModel](saved_model.ipynb) y [Checkpoint](checkpoint_saved.ipynb)).

## Preparación

Empieza con imports y un conjunto de datos sencillo:

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [[0.8], [0.9], [1.]]

## TensorFlow 1: Entrenar y evaluar con tf.estimator.Estimator

Este ejemplo muestra cómo realizar el entrenamiento y la evaluación con `tf.estimator.Estimator` en TensorFlow 1.

Empieza definiendo unas cuantas funciones: una función de entrada para los datos de entrenamiento, una función de entrada de evaluación para los datos de evaluación y una función de modelo que indique al `Estimator` cómo se define la op de entrenamiento con las características y las etiquetas:

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

def _eval_input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(1)(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  return tf1.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

Instancia tu `Estimator`, y entrena el modelo:

In [ ]:
estimator = tf1.estimator.Estimator(model_fn=_model_fn)
estimator.train(_input_fn)

Evalúa el programa con el conjunto de evaluación:

In [ ]:
estimator.evaluate(_eval_input_fn)

## TensorFlow 2: Entrenar y evaluar con los métodos Keras incorporados

Este ejemplo muestra cómo realizar el entrenamiento y la evaluación con `Model.fit` y `Model.evaluate` de Keras en TensorFlow 2. (Puedes obtener más información en la guía [Entrenamiento y evaluación con los métodos incorporados](https://www.tensorflow.org/guide/keras/train_and_evaluate).)

- Para empezar, prepara la canalización del conjunto de datos con las API `tf.data.Dataset`.
- Define un modelo Keras sencillo [Sequential](https://www.tensorflow.org/guide/keras/sequential_model) con una capa lineal (`tf.keras.layers.Dense`).
- Instancia un optimizador Adagrad (`tf.keras.optimizers.Adagrad`).
- Configura el modelo para el entrenamiento pasando la variable `optimizer` y la pérdida de error cuadrático medio (`"mse"`) a `Model.compile`.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer=optimizer, loss="mse")

Una vez hecho esto, ya puedes entrenar el modelo llamando a `Model.fit`:

In [ ]:
model.fit(dataset)

Por último, evalúa el modelo con `Model.evaluate`:

In [ ]:
model.evaluate(eval_dataset, return_dict=True)

## TensorFlow 2: Entrena y evalúa con un paso de entrenamiento personalizado y métodos Keras integrados

En TensorFlow 2, también puedes escribir tu propia función de pasos de entrenamiento personalizada con `tf.GradientTape` para realizar pasadas hacia delante y hacia atrás, sin dejar de aprovechar el soporte de entrenamiento incorporado, como `tf.keras.callbacks.Callback` y `tf.distribute.Strategy`. (Más información en [Cómo personalizar lo que ocurre en Model.fit](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit) y [Cómo escribir bucles de entrenamiento personalizados desde cero](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch)).

En este ejemplo, empieza por crear un `tf.keras.Model` personalizado subclasificando `tf.keras.Sequential` que anule `Model.train_step` (aprende más sobre [hacer subclases de tf.keras.Model](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). Dentro de esa clase, define una función personalizada `train_step` que, para cada lote de datos, realice una pasada hacia delante y otra hacia atrás durante un paso de entrenamiento.


In [ ]:
class CustomModel(tf.keras.Sequential):
  """A custom sequential model that overrides `Model.train_step`."""

  def train_step(self, data):
    batch_data, labels = data

    with tf.GradientTape() as tape:
      predictions = self(batch_data, training=True)
      # Compute the loss value (the loss function is configured
      # in `Model.compile`).
      loss = self.compiled_loss(labels, predictions)

    # Compute the gradients of the parameters with respect to the loss.
    gradients = tape.gradient(loss, self.trainable_variables)
    # Perform gradient descent by updating the weights/parameters.
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
    # Update the metrics (includes the metric that tracks the loss).
    self.compiled_metrics.update_state(labels, predictions)
    # Return a dict mapping metric names to the current values.
    return {m.name: m.result() for m in self.metrics}

Luego, como antes:

- Prepara la canalización del conjunto de datos con `tf.data.Dataset`.
- Define un modelo sencillo con una capa `tf.keras.layers.Dense`.
- Instancia Adagrad (`tf.keras.optimizers.Adagrad`)
- Configura el modelo para el entrenamiento con `Model.compile`, usando el error cuadrático medio (`"mse"`) como función de pérdida.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

model = CustomModel([tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer=optimizer, loss="mse")

Llama a `Model.fit` para entrenar el modelo:

In [ ]:
model.fit(dataset)

Y, por último, evalúa el programa con `Model.evaluate`:

In [ ]:
model.evaluate(eval_dataset, return_dict=True)

## Siguientes pasos

Recursos adicionales de Keras que puedes encontrar útiles:

- Guía: [Entrenamiento y evaluación con los métodos incorporados](https://www.tensorflow.org/guide/keras/train_and_evaluate)
- Guía: [Personalice lo que ocurre en Model.fit](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit)
- Guía: [Escribir un bucle de entrenamiento desde cero](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch)
- Guía: [Cómo crear nuevas capas y modelos Keras mediante creación de subclases](https://www.tensorflow.org/guide/keras/custom_layers_and_models)

Las siguientes guías pueden ser útiles para migrar los flujos de trabajo de la estrategia de distribución desde APIs `tf.estimator`:

- [Migrar de TPUEstimator a TPUStrategy](tpu_estimator.ipynb)
- [Migrar el entrenamiento monotrabajador con múltiples GPUs](mirrored_strategy.ipynb)
- [Migrar el entrenamiento multitrabajador con CPU/GPU](multi_worker_cpu_gpu_training.ipynb)